# Programmatic use of MOCs in Python

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import math

## Retrieving the various MOCs

In [ ]:
from astroquery.cds import cds
from mocpy import MOC

In [ ]:
isophot = MOC.from_url('http://cds.unistra.fr/adass2018/ISOPHOTHiPS/Moc.fits')

`isophot` contains the MOC associated to the ISOPHOT HiPS we have generated.

In [ ]:
hstv = cds.find_datasets("ID=CDS/P/HST/V", return_moc=True)
galex = cds.find_datasets("ID=CDS/P/GALEXGR6/AIS/color", return_moc=True)

`hstv` and `galex` contain the MOCs retrieved from the CDS server. 

## Computing the intersection

One can use these MOCs e.g. for computing their intersection. This will give us a new `mocpy.MOC` instance with the same HEALPix order as the best one (HST in our case).

In [ ]:
moc = galex.intersection(hstv).intersection(isophot)
moc.max_order

The area of the intersection is available with `moc.sky_fraction` (converted below in square degrees)

In [ ]:
moc.sky_fraction*360**2/math.pi

## Visualization

`mocpy.MOC` class has a `fill` method responsible for plotting the HEALPix cells of a MOC on a predefined `matplotlib` axe. This method accepts an `astropy.wcs.WCS` object along with a `matplotlib.axes.Axes` and some `matplotlib` style keyword arguments.

The next cell of code will define a `matplotlib` context and use it for drawing the various MOCs along with their intersection MOC.


In [ ]:
import matplotlib.pyplot as plt
from mocpy import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
# MOCPy offers a way to easily create an WCS focusing in a specific skycoord with a
# FOV. The WCS class embeds internally a astropy.wcs.WCS object
# This define an ICRS aitoff projection.
fig = plt.figure(111, figsize=(15, 15))
with WCS(fig,
        fov=50 * u.deg,
        center=SkyCoord(300, 35, unit='deg', frame='icrs'),
        coordsys="icrs",
        rotation=Angle(0, u.degree),
        projection="AIT") as wcs:
    # Create an mpl axe
    ax = fig.add_subplot(1, 1, 1, projection=wcs)
    # Calls to `mocpy.MOC.fill` for each of the three MOCs with various mpl styling keywords
    # to differentiate them from each other.
    galex.fill(ax=ax, wcs=wcs, edgecolor='g', facecolor='g', linewidth=1.0, fill=True, alpha=0.5, label='P/GALEXGR6/AIS/color')
    isophot.fill(ax=ax, wcs=wcs, edgecolor='y', facecolor='y', linewidth=1.0, fill=True, alpha=0.5, label='ISOPHOT tutorial')
    hstv.fill(ax=ax, wcs=wcs, edgecolor='r', facecolor='r', linewidth=1.0, fill=True, alpha=0.5, label='P/HST/V')
    moc.fill(ax=ax, wcs=wcs, edgecolor='b', facecolor='b', linewidth=1.0, fill=True, alpha=0.5, label='intersection')


plt.xlabel('ra')
plt.ylabel('dec')
plt.title('Several MOCs and their intersection')
plt.grid(color="black", linestyle="dotted")
plt.legend(loc='upper left')
plt.show()
plt.close()

## Finding relevant catalogues

Querying the MOCServer using `astroquery.cds` can help us knowing which vizier tables have sources in our intersection MOC. As `astroquery.cds.query_region` returns an `astropy.table.Table`, it is also possible to do some post-filtering on the large amount of tables we might receive.

In [ ]:
data_cols = cds.query_region(moc)

As the returned object is an `astropy.table.Table`, one can use mask arrays to filter the table. We add constraints to only keep catalogues, with observation in the Infrared, with at least 10,000 rows.

In [ ]:
mask = (data_cols['obs_regime'] == 'Infrared') & \
    (data_cols['dataproduct_type'] == 'catalog') & \
    (data_cols['nb_rows'] > 10000) 
col = data_cols[mask]['obs_id', 'obs_title', 'dataproduct_type', 'cs_service_url', 'nb_rows']
col

## View in Aladin Lite

To finish this tutorial, we will use `ipyaladin` to display in Aladin Lite some MOCs:

In [ ]:
import ipyaladin as ipyal
aladin= ipyal.Aladin(target='19 40 +15', fov=5)
aladin

In [ ]:
# Add the ISOPHOT and intersection MOCs to the view
aladin.add_moc_from_dict(isophot.serialize(format='json'), {
     'opacity': 0.5,
     'color': "#e6e600",
})
aladin.add_moc_from_dict(moc.serialize(format='json'), {
     'opacity': 0.8,
     'color': "#FF0000",
})

And we overlay sources from VizieR catalogue `V/114/msx6_s` found in the previous query.

In [ ]:
from astroquery.vizier import Vizier

v = Vizier(columns=['*', '_RAJ2000', '_DEJ2000'])
v.ROW_LIMIT = 2000
msx6 = v.get_catalogs('V/114/msx6_s')[0]
aladin.add_table(msx6)

We can also query directly a catalogue (even a very large one like 2MASS - II/246/out) for sources located inside a MOC:

In [ ]:
twomass = moc.query_vizier_table(table_id='II/246/out')

And load it in Aladin Lite

In [ ]:
aladin.add_table(twomass)